# Install packages and download dataset

In [ ]:
# https://huggingface.co/google-bert/bert-base-multilingual-cased

In [ ]:
!pip install transformers torch
!pip install datasets

In [ ]:
!pip install accelerate -U

# Build a new dataset with description and reference

In [ ]:
# Download EXIST dataset
# Please manually download and extract the EXIST_dataset.zip to ./data/ directory
# The dataset should contain GPT descriptions and initial answers

Downloading...
From: https://drive.google.com/uc?id=1p4icEWmTXic35fMUQ7jmjEf89-Vnwvgn
To: /content/EXIST_dataset.zip
100% 1.20M/1.20M [00:00<00:00, 29.8MB/s]
Archive:  EXIST_dataset.zip
   creating: /content/EXIST_dataset/
  inflating: /content/EXIST_dataset/Spanish_Meme_GPT_Initial_Answers_test.json  
  inflating: /content/__MACOSX/EXIST_dataset/._Spanish_Meme_GPT_Initial_Answers_test.json  
  inflating: /content/EXIST_dataset/English_Meme_GPT_Initial_Answers_test.json  
  inflating: /content/__MACOSX/EXIST_dataset/._English_Meme_GPT_Initial_Answers_test.json  
  inflating: /content/EXIST_dataset/Spanish_Meme_GPT_Initial_Answers_test_training.json  
  inflating: /content/__MACOSX/EXIST_dataset/._Spanish_Meme_GPT_Initial_Answers_test_training.json  
  inflating: /content/EXIST_dataset/Spanish_Meme_GPT_Description_training.json  
  inflating: /content/__MACOSX/EXIST_dataset/._Spanish_Meme_GPT_Description_training.json  
  inflating: /content/EXIST_dataset/English_Meme_GPT_Description_tr

In [ ]:
# Download EXIST 2024 Memes Dataset
# Please manually download and extract the EXIST_2024_Memes_Dataset.zip to ./data/ directory

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/110601.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._110601.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/210654.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._210654.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/211946.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._211946.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/111440.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._111440.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/211415.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._211415.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/111010.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._111010.jp

In [ ]:
original_training_path = './data/EXIST 2024 Memes Dataset/training/EXIST2024_training.json'

In [ ]:
GPT_english_path = './data/EXIST_dataset/English_Meme_GPT_Description_training.json'
GPT_spanish_path = './data/EXIST_dataset/Spanish_Meme_GPT_Description_training.json'

In [ ]:
import json

def build_datasets(input_path_en, input_path_sp, input_path_original, output_path):
    with open(input_path_en, 'r') as file:
        data_en = json.load(file)
    with open(input_path_sp, 'r') as file:
        data_sp = json.load(file)
    with open(input_path_original, 'r') as file:
        data_original = json.load(file)

    for key, value in data_en.items():
        image_id = key.split('.')[0]
        if image_id in data_original:
            data_original[image_id]['description'] = value['description']
            data_original[image_id]['reference'] = value['reference']
        else:
            print(f'EN ID {image_id} not found in the original file.')

    for key, value in data_sp.items():
        image_id = key.split('.')[0]
        if image_id in data_original:
            data_original[image_id]['description'] = value['description']
            data_original[image_id]['reference'] = value['reference']
        else:
            print(f'SP ID {image_id} not found in the original file.')

    with open(output_path, 'w') as file:
        json.dump(data_original, file, ensure_ascii=False, indent=4)

    print("Data has been merged and saved in", output_path)


In [ ]:
build_datasets(GPT_english_path, GPT_spanish_path, original_training_path, "./data/new_dataset_training.json")

Data has been merged and saved in new_dataset_training.json


# Split english memes and spanish memes to training and test dataset seperately

In [ ]:
# Split the json file to english and spanish seperately
def split_json_by_key(input_path, output_path_spanish, output_path_english):
    # 读取原始JSON文件
    with open(input_path, 'r') as file:
        data = json.load(file)

    # 初始化两个字典来存储分类后的数据
    spanish_memes = {}
    english_memes = {}

    # 遍历原始数据，根据key的首位数字进行分类
    for key, value in data.items():
        if key.startswith('1'):
            spanish_memes[key] = value
        elif key.startswith('2'):
            english_memes[key] = value

    # 写入Spanish memes数据到新的JSON文件
    with open(output_path_spanish, 'w', encoding='utf-8') as file:
        json.dump(spanish_memes, file, ensure_ascii=False, indent=4)

    # 写入English memes数据到新的JSON文件
    with open(output_path_english, 'w', encoding='utf-8') as file:
        json.dump(english_memes, file, ensure_ascii=False, indent=4)

    print("Data has been successfully split and saved.")

In [ ]:
split_json_by_key('./data/new_dataset_training.json', './data/spanish_memes.json', './data/english_memes.json')

Data has been successfully split and saved.


In [ ]:
# Download ground truth data
# Please manually download EXIST2024_training_task4_gold_hard.json to ./data/ directory

Downloading...
From: https://drive.google.com/uc?id=14lpl1vUz4kDIOCxZ4sYGPVJrhTZ3CsSf
To: /content/EXIST2024_training_task4_gold_hard.json
100% 245k/245k [00:00<00:00, 34.1MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json

# 将gold数据和现有的数据结合
def process_data(main_json_path, ground_truth_json_path, test_size=0.2):
    # 从文件中读取JSON数据
    with open(main_json_path, 'r') as file:
        main_data = json.load(file)
    with open(ground_truth_json_path, 'r') as file:
        ground_truth_data = json.load(file)

    # 转换列表中的字典为DataFrame
    main_df = pd.DataFrame.from_dict(main_data, orient='index')
    ground_truth_df = pd.DataFrame(ground_truth_data)

    # 假设ground_truth_df的数据已正确设置id为索引
    ground_truth_df.set_index('id', inplace=True)
    ground_truth_df.rename(columns={'value': 'gold'}, inplace=True)

    # 合并两个 DataFrame
    complete_df = main_df.join(ground_truth_df, how='left', rsuffix='_gt')

    # 返回处理后的训练集和测试集
    return complete_df

In [ ]:
# 调用这个函数处理数据
# 首先分开处理english和spanish数据集
spanish_df = process_data('./data/spanish_memes.json', './data/EXIST2024_training_task4_gold_hard.json')
english_df = process_data('./data/english_memes.json', './data/EXIST2024_training_task4_gold_hard.json')

In [ ]:
english_df

,id_EXIST,lang,text,meme,path_memes,number_annotators,annotators,gender_annotators,age_annotators,ethnicities_annotators,study_levels_annotators,countries_annotators,labels_task4,labels_task5,labels_task6,split,description,reference,test_case,gold
210001,210001,en,I can't believe she cheated on me for another ...,210001.jpeg,memes/210001.jpeg,6,"[Annotator_457, Annotator_458, Annotator_459, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Poland, United Kingdom, United Kingdom, Austr...","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN-MEME_EN,The meme shows a man appearing distressed with...,YES,EXIST2024,NO
210002,210002,en,"When your homie says ""a woman's body count sho...",210002.jpeg,memes/210002.jpeg,6,"[Annotator_457, Annotator_458, Annotator_459, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Poland, United Kingdom, United Kingdom, Austr...","[YES, YES, NO, YES, NO, YES]","[DIRECT, JUDGEMENTAL, -, DIRECT, -, JUDGEMENTAL]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",TRAIN-MEME_EN,The meme shows two basketball players looking ...,YES,EXIST2024,YES
210003,210003,en,"Yes, cocks are important, I'm afraid *********...",210003.jpeg,memes/210003.jpeg,6,"[Annotator_457, Annotator_458, Annotator_459, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Poland, United Kingdom, United Kingdom, Austr...","[NO, YES, NO, NO, NO, YES]","[-, DIRECT, -, -, -, JUDGEMENTAL]","[[-], [IDEOLOGICAL-INEQUALITY, STEREOTYPING-DO...",TRAIN-MEME_EN,The meme features a character from Game of Thr...,YES,EXIST2024,NO
210004,210004,en,2016 TODAYS SOCIETY #MGTOW Please go out with ...,210004.jpeg,memes/210004.jpeg,6,"[Annotator_457, Annotator_458, Annotator_459, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Poland, United Kingdom, United Kingdom, Austr...","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, JUDGEMENTAL, ...","[[STEREOTYPING-DOMINANCE], [IDEOLOGICAL-INEQUA...",TRAIN-MEME_EN,"The meme contrasts two images, one depicting a...",YES,EXIST2024,YES
210005,210005,en,The official friend zone logo IG: @memes_pl0x ...,210005.jpeg,memes/210005.jpeg,6,"[Annotator_457, Annotator_458, Annotator_459, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Poland, United Kingdom, United Kingdom, Austr...","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, DIRECT]","[[-], [-], [-], [-], [-], [IDEOLOGICAL-INEQUAL...",TRAIN-MEME_EN,The meme shows a photo of a young man and woma...,YES,EXIST2024,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212006,212006,en,u gon act like a bitch u gon die like a bitch,212006.jpeg,memes/212006.jpeg,6,"[Annotator_883, Annotator_884, Annotator_885, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[Black or African American, White or Caucasian...","[Bachelor’s degree, Bachelor’s degree, Bachelo...","[South Africa, Poland, Canada, Poland, Italy, ...","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, JUDGEMENTAL, JUD...","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",TRAIN-MEME_EN,The meme features two images of a sloth with c...,YES,EXIST2024,YES
212007,212007,en,SHE LOOKS LIKE EVERY OTHER BITCH LIKE makeamem...,212007.jpeg,memes/212007.jpeg,6,"[Annotator_883, Annotator_884, Annotator_885, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[Black or African American, White or

In [ ]:
spanish_df.head()

,id_EXIST,lang,text,meme,path_memes,number_annotators,annotators,gender_annotators,age_annotators,ethnicities_annotators,study_levels_annotators,countries_annotators,labels_task4,labels_task5,labels_task6,split,description,reference,test_case,gold
110001,110001,es,2+2=5 MITO Albert Einstein tenía bajo rendimie...,110001.jpeg,memes/110001.jpeg,6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 18-22, 23-45]","[Hispano or Latino, Hispano or Latino, Hispano...","[High school degree or equivalent, Master’s de...","[Mexico, Spain, Argentina, Spain, Mexico, Mexico]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN-MEME_ES,"El meme muestra dos paneles. En el primero, ba...",YES,EXIST2024,YES
110002,110002,es,CUANDO UNA MUJER VA A LUCHAR POR SUS DERECHOS,110002.jpeg,memes/110002.jpeg,6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 18-22, 23-45]","[Hispano or Latino, Hispano or Latino, Hispano...","[High school degree or equivalent, Master’s de...","[Mexico, Spain, Argentina, Spain, Mexico, Mexico]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, JUDGE...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN-MEME_ES,El meme muestra una imagen de varios hombres d...,YES,EXIST2024,YES
110003,110003,es,ІЯ ЕГЕЯ Е MOA ¿El Partido Republicano busca pe...,110003.jpeg,memes/110003.jpeg,6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 18-22, 23-45]","[Hispano or Latino, Hispano or Latino, Hispano...","[High school degree or equivalent, Master’s de...","[Mexico, Spain, Argentina, Spain, Mexico, Mexico]","[YES, YES, NO, NO, NO, NO]","[DIRECT, DIRECT, -, -, -, -]","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN-MEME_ES,El meme muestra una imagen de un hombre sonrie...,YES,EXIST2024,NO
110004,110004,es,"Paises que ""apoyan"" los derechos de la mujer A...",110004.jpeg,memes/110004.jpeg,6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 18-22, 23-45]","[Hispano or Latino, Hispano or Latino, Hispano...","[High school degree or equivalent, Master’s de...","[Mexico, Spain, Argentina, Spain, Mexico, Mexico]","[YES, YES, NO, NO, YES, NO]","[JUDGEMENTAL, JUDGEMENTAL, -, -, JUDGEMENTAL, -]","[[IDEOLOGICAL-INEQUALITY], [IDEOLOGICAL-INEQUA...",TRAIN-MEME_ES,El meme muestra un mapa del mundo coloreado pa...,YES,NaN,NaN
110005,110005,es,Ya verás como este 8 de marzo hay uno que te s...,110005.jpeg,memes/110005.jpeg,6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 18-22, 23-45]","[Hispano or Latino, Hispano or Latino, Hispano...","[High school degree or equivalent, Master’s de...","[Mexico, Spain, Argentina, Spain, Mexico, Mexico]","[NO, YES, NO, NO, YES, NO]","[-, JUDGEMENTAL, -, -, DIRECT, -]","[[-], [IDEOLOGICAL-INEQUALITY], [-], [-], [IDE...",TRAIN-MEME_ES,El meme muestra a cuatro mujeres riendo juntas...,NO,EXIST2024,NO


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def clean_df(df):

    # 映射 'gold' 列并立即删除任何未映射的值（即映射之外的值导致的 NaN）
    label_mapping = {'YES': 1, 'NO': 0}
    df['gold'] = df['gold'].map(label_mapping, na_action='ignore')  # 使用 na_action='ignore' 保留 NaNs 以便于检测
    df.dropna(subset=['gold'], inplace=True)  # 删除映射后仍为 NaN 的行

    # 确保 text 和 description 列为字符串类型
    df['text'] = df['text'].astype(str)
    df['description'] = df['description'].astype(str)

    # 合并 'text' 和 'description' 列
    df['combined_text'] = df['text'] + " " + df['description']

    # 提取特征和标签
    X = df['combined_text'].values
    y = df['gold'].values.astype(int)  # 转换为整数
    # # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Fine-tuning using mBERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import EarlyStoppingCallback

In [ ]:
from datasets import load_metric
import numpy as np

f1_metric = load_metric("f1")
accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    f1_result = f1_metric.compute(predictions=predictions, references=labels, average='binary')

    accuracy_result = accuracy_metric.compute(predictions=predictions, references=labels)

    return {
        "f1": f1_result['f1'],
        "accuracy": accuracy_result['accuracy']
    }

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
X_train_spanish,X_test_spanish, y_train_spanish, y_test_spanish = clean_df(spanish_df)

In [ ]:
print("Number of entries in X_train_spanish:", len(X_train_spanish))
print("Number of entries in X_test_spanish:", len(X_test_spanish))
print("Number of entries in y_train_spanish:", len(y_train_spanish))
print("Number of entries in y_test_spanish:", len(y_test_spanish))

Number of entries in X_train_spanish: 1369
Number of entries in X_test_spanish: 343
Number of entries in y_train_spanish: 1369
Number of entries in y_test_spanish: 343


In [ ]:
X_train_english, X_test_english, y_train_english, y_test_english = clean_df(english_df)

In [ ]:
print("Number of entries in X_train_english:", len(X_train_english))
print("Number of entries in X_test_english:", len(X_test_english))
print("Number of entries in y_train_english:", len(y_train_english))
print("Number of entries in y_test_english:", len(y_test_english))

Number of entries in X_train_english: 1366
Number of entries in X_test_english: 342
Number of entries in y_train_english: 1366
Number of entries in y_test_english: 342


In [ ]:
# Concatenate training features (texts) from both languages
X_train_combined = np.concatenate((X_train_spanish, X_train_english))
X_test_combined = np.concatenate((X_test_spanish, X_test_english))

# Concatenate training labels from both languages
y_train_combined = np.concatenate((y_train_spanish, y_train_english))
y_test_combined = np.concatenate((y_test_spanish, y_test_english))

print("Combined X_train size:", len(X_train_combined))
print("Combined X_test size:", len(X_test_combined))
print("Combined y_train size:", len(y_train_combined))
print("Combined y_test size:", len(y_test_combined))

Combined X_train size: 2735
Combined X_test size: 685
Combined y_train size: 2735
Combined y_test size: 685


In [ ]:
import numpy as np

np.savetxt(f'./data/english_train_X.txt', X_train_english, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/english_test_X.txt', X_test_english, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/english_train_y.txt', y_train_english, fmt='%d')
np.savetxt(f'./data/english_test_y.txt', y_test_english, fmt='%d')

In [ ]:
np.savetxt(f'./data/spanish_train_X.txt', X_train_spanish, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/spanish_test_X.txt', X_test_spanish, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/spanish_train_y.txt', y_train_spanish, fmt='%d')
np.savetxt(f'./data/spanish_test_y.txt', y_test_spanish, fmt='%d')

In [ ]:
np.savetxt(f'./data/combined_train_X.txt', X_train_combined, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/combined_test_X.txt', X_test_combined, fmt='%s', encoding='utf-8')
np.savetxt(f'./data/combined_train_y.txt', y_train_combined, fmt='%d')
np.savetxt(f'./data/combined_test_y.txt', y_test_combined, fmt='%d')

In [ ]:
tokenizer_mbert = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_mbert = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model_mbert.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

### Fine-tune on the Spanish dataset

In [ ]:
train_dataset = TextDataset(X_train_spanish, y_train_spanish, tokenizer_mbert)
test_dataset = TextDataset(X_test_spanish, y_test_spanish, tokenizer_mbert)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-5,
    max_grad_norm=1.0,
    load_best_model_at_end=True
)


trainer = Trainer(
    model=model_mbert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.637200,0.656334,0.763964,0.618076
2,0.646000,0.625689,0.762452,0.638484
3,0.627900,0.659159,0.772894,0.638484
4,0.566600,0.649643,0.732026,0.641399
5,0.439700,0.698070,0.694789,0.641399


F1: {'f1': 0.7639639639639639}
Accuracy: {'accuracy': 0.6180758017492711}
F1: {'f1': 0.7624521072796935}
Accuracy: {'accuracy': 0.6384839650145773}
F1: {'f1': 0.7728937728937728}
Accuracy: {'accuracy': 0.6384839650145773}
F1: {'f1': 0.7320261437908497}
Accuracy: {'accuracy': 0.641399416909621}
F1: {'f1': 0.694789081885856}
Accuracy: {'accuracy': 0.641399416909621}


TrainOutput(global_step=430, training_loss=0.5858863087587578, metrics={'train_runtime': 185.6201, 'train_samples_per_second': 36.876, 'train_steps_per_second': 2.317, 'total_flos': 1646222151178800.0, 'train_loss': 0.5858863087587578, 'epoch': 5.0})

### Fine-tune on the English dataset

In [ ]:
train_dataset = TextDataset(X_train_english, y_train_english, tokenizer_mbert)
test_dataset = TextDataset(X_test_english, y_test_english, tokenizer_mbert)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-5,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
)

trainer = Trainer(
    model=model_mbert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.677200,0.671220,0.674473,0.593567
2,0.626000,0.606538,0.699229,0.657895
3,0.539100,0.606455,0.750000,0.690058
4,0.596800,0.543869,0.786408,0.742690
5,0.458100,0.584613,0.739645,0.742690


F1: {'f1': 0.674473067915691}
Accuracy: {'accuracy': 0.5935672514619883}
F1: {'f1': 0.699228791773779}
Accuracy: {'accuracy': 0.6578947368421053}
F1: {'f1': 0.7499999999999999}
Accuracy: {'accuracy': 0.6900584795321637}
F1: {'f1': 0.7864077669902911}
Accuracy: {'accuracy': 0.7426900584795322}
F1: {'f1': 0.7396449704142012}
Accuracy: {'accuracy': 0.7426900584795322}


TrainOutput(global_step=430, training_loss=0.5607916765434797, metrics={'train_runtime': 192.9318, 'train_samples_per_second': 35.401, 'train_steps_per_second': 2.229, 'total_flos': 1797048508108800.0, 'train_loss': 0.5607916765434797, 'epoch': 5.0})

# Fine-tune using XLM-RoBERTa

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

tokenizer_xlm = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model_xlm = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)
model_xlm.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

### Fine-tune on Spanish dataset

In [ ]:
train_dataset = TextDataset(X_train_spanish, y_train_spanish, tokenizer_xlm)
test_dataset = TextDataset(X_test_spanish, y_test_spanish, tokenizer_xlm)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-5,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
)

trainer = Trainer(
    model=model_xlm,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.703100,0.696828,0.000000,0.381924
2,0.683000,0.675971,0.763964,0.618076
3,0.676900,0.653367,0.763964,0.618076
4,0.653600,0.660431,0.763964,0.618076
5,0.634500,0.595848,0.769841,0.661808


F1: {'f1': 0.0}
Accuracy: {'accuracy': 0.3819241982507289}
F1: {'f1': 0.7639639639639639}
Accuracy: {'accuracy': 0.6180758017492711}
F1: {'f1': 0.7639639639639639}
Accuracy: {'accuracy': 0.6180758017492711}
F1: {'f1': 0.7639639639639639}
Accuracy: {'accuracy': 0.6180758017492711}
F1: {'f1': 0.7698412698412698}
Accuracy: {'accuracy': 0.6618075801749271}


TrainOutput(global_step=215, training_loss=0.6731603522633397, metrics={'train_runtime': 181.5559, 'train_samples_per_second': 37.702, 'train_steps_per_second': 1.184, 'total_flos': 1424615323135500.0, 'train_loss': 0.6731603522633397, 'epoch': 5.0})

### Fine-tune on English dataset

In [ ]:
train_dataset = TextDataset(X_train_english, y_train_english, tokenizer_xlm)
test_dataset = TextDataset(X_test_english, y_test_english, tokenizer_xlm)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-5,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
)

trainer = Trainer(
    model=model_xlm,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.611400,0.580390,0.735751,0.701754
2,0.622500,0.595148,0.753117,0.710526
3,0.512400,0.578199,0.780000,0.742690
4,0.582600,0.552537,0.764398,0.736842
5,0.425800,0.580199,0.760563,0.751462


F1: {'f1': 0.7357512953367875}
Accuracy: {'accuracy': 0.7017543859649122}
F1: {'f1': 0.7531172069825436}
Accuracy: {'accuracy': 0.7105263157894737}
F1: {'f1': 0.78}
Accuracy: {'accuracy': 0.7426900584795322}
F1: {'f1': 0.7643979057591623}
Accuracy: {'accuracy': 0.7368421052631579}
F1: {'f1': 0.7605633802816901}
Accuracy: {'accuracy': 0.7514619883040936}


TrainOutput(global_step=430, training_loss=0.5454334807950397, metrics={'train_runtime': 217.3434, 'train_samples_per_second': 31.425, 'train_steps_per_second': 1.978, 'total_flos': 1754930183700000.0, 'train_loss': 0.5454334807950397, 'epoch': 5.0})